## Question 3 :
Implémentez en utilisant python3 un connecteur Facebook et/ou Instagram
permettant de colleter des postes (image, texte et commentaires liés aux images) par
rapport à un sujet défini, exemple « le décès du président Jacques Chirac ».
Il est recommandé de stocker les textes et les images dans une base MongoDB.
Le but de cette question est d’évaluer la qualité de votre code source.

## Selenium
Selenium permet de scraper des publication facebook en relation avec un Hashtag spécifique

pour configurer Selenium avec ChromeDriver sur Ubuntu 20.04 
https://tecadmin.net/setup-selenium-chromedriver-on-ubuntu/

In [3]:
from selenium import webdriver 
from pymongo import MongoClient
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pymongo
import requests
from time import sleep
import urllib.request
import os

In [2]:
# Pilote selenium Chrome 
wd = webdriver.Chrome(r'chromedriver')

In [3]:
# Go to the url
wd.get('https://www.facebook.com/')## i will not use the normal fb because i found a problem with the search button 
wd.get('https://mbasic.facebook.com/')



In [4]:
#target username and password
username = WebDriverWait(wd, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(wd, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

#enter username and password
username.clear()
username.send_keys("")#add adresse mail facebook
password.clear()
password.send_keys("")#add password

#target the login button and click it
login_btn = wd.find_element(By.XPATH,'//*[@id="login_form"]/ul/li[3]/input')
login_btn.click()

In [5]:


#Skip remember popup
login_btn = wd.find_element(By.XPATH,'//*[@id="root"]/table/tbody/tr/td/div/form/div/input')
login_btn.click()

In [6]:
#Search
search = WebDriverWait(wd, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='query']")))

#enter desired Hashtag
search.clear()
search.send_keys("#le décès du président Jacques Chirac")

#search button click
search_btn = wd.find_element(By.XPATH,'//*[@id="header"]/form/table/tbody/tr/td[3]/input')
search_btn.click()


## BeautifulSoup
 parser les pages html et extraire des données

In [7]:
list_dir=""
default_dir = os.path.join(list_dir,"collection")
opener = urllib.request.build_opener()
urllib.request.install_opener(opener)

In [25]:
posts = wd.find_element(By.XPATH,'//*[@id="u_0_8_H6"]/a')
links= posts.find_elements(By.LINK_TEXT,'All')

pubs = []
for link in links:
 pub = {}
 page_content = requests.get(link.get_attribute('href')).content
 soup = BeautifulSoup(page_content, 'html.parser')
    #get post text
    
 pub['text'] = soup.p.text
 pub['image'] = soup.img["src"]
 img_url = soup.img["src"]
 filename = os.path.join(default_dir, img_url.split("/")[-1])
 img_data = opener.open(img_url)
 f = open(filename,"wb")
 f.write(img_data.read())
 f.close()
 pubs.append(pub)

In [26]:
pubs

[]

## MongoDB
 stocker les textes et les images dans une base MongoDB.

In [27]:
#MongoDB connection
import pymongo
from pymongo import MongoClient
client = pymongo.MongoClient("mongodb://127.0.0.1:27017")


In [70]:
mydb = client["facebook"]
mycol = mydb["posts"]
mydb


Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'facebook')

In [76]:
#insert the list of dicts
mycol.insert_many(pubs)

#Show a sample 
mycol.find_one()